<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/pull_one_hr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
!pip install censusgeocode
!pip install rets
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys
from rets import Session
import datetime
from datetime import datetime,timedelta
!pip install pytz
import pytz



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [117]:
login_url = 'https://ntrdd.mlsmatrix.com/rets/Login.ashx' 
username = '0671181_NID'
password = 'Rt$tg6jx'
rets_client = Session(login_url, username, password)
rets_client.login()

#filter = {
       # "MlsStatus": "SLD",
       # "ModificationTimestamp": {"$gte": dt_string}
        #}
#result = rets_client.search(resource='Property', resource_class='Property', limit = 10,search_filter=filter)





In [ ]:
# low price
tz_IN = pytz.timezone('US/Central')
ini_time_for_now = datetime.now(tz_IN)
today_before_1hours = ini_time_for_now - timedelta(hours = 1)
dt_string = today_before_1hours.strftime("%Y-%m-%dT%H:%M:%S.000")


lowerPrice_result = rets_client.search(resource = 'Property', resource_class='Property', dmql_query='(ModificationTimestamp = {}+),(MlsStatus = Act)'.format(dt_string))


for item in lowerPrice_result:
  if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence' and item['PreviousListPrice']>item['ListPrice']):
    df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True)
column = []
for key in lowerPrice_result:
    column.append(key)
    df.columns = column
    # add a column: (PreviousListPrice - ListPrice) / ListPrice

df


AttributeError: ignored

In [212]:
#Relist - Back on Market

tz_IN = pytz.timezone('US/Central')
ini_time_for_now = datetime.now(tz_IN)
today_before_1hours = ini_time_for_now - timedelta(hours = 1)
dt_string = today_before_1hours.strftime("%Y-%m-%dT%H:%M:%S.000")

relist_result = rets_client.search(resource = 'Property', resource_class='Property', dmql_query='(ModificationTimestamp = {}+),(MlsStatus = Act)'.format(dt_string))

df = pd.DataFrame()
for item in relist_result:
    if(item['PropertyType'] == 'Residential' and item['PropertySubType'] == 'Single Family Residence' and (item['PreviousStatus'] == "AOC" or item['PreviousStatus'] == "Expired" or item['PreviousStatus'] == "Active Option Contract")):
      df = pd.concat([df, pd.DataFrame(item.values())], axis=1, ignore_index=True)
column = []
for key in lowerPrice_result:
    column.append(key)
    df.columns = column

df   


,0,1,2,3,4,5,6,7,8,9
0,163733811,415368859,415574769,416011455,416280632,416301280,416570057,416611541,416647016,416878584
1,2,2,1,2,2,3,4,4,4,2
2,0,0,0,1,0,0,1,0,2,0
3,2,2,1,3,2,3,5,4,6,2
4,3,3,2,4,3,4,5,4,5,3
...,...,...,...,...,...,...,...,...,...,...
423,,,,,,,,,,
424,,1,,1,,1,1,1,1,
425,,,,,,,,,,
426,One,One,One,One,One,One,Two,Two,Two,One
